# Upper Confidence Bounds (A k-armed bandit problem)
by __Shivangi Agarwal and Sandeep Banik__ | Reinforcement Learning

The epsilon greedy provides a method to choose possible actions encouraging exploration. However while choosing the actions at any given time, the method is unbiased and chooses any action with equal probability. One would expect that if an arm/action is choosen multiple times, its estimate is much better as compared to other arms/actions. This is exactly captured by the upper confidence bounds.

<img src="images\UCB-intro.png">

<img src="images\UCB-define.png">

The equation to choose an action is given by, 

$A_{t} = arg \max_{a} \left [ Q_{t}(a) + c\sqrt{\frac{ln(t)}{N_{t}(a)}} \right ] $

$N_t(a)$ is number of times the action $a$ is choosen, which reduces the uncertainity as is present in the denominator. The numerator term, $ln(t)$ keeps increasing with each time step. However, the increase is logarithmic in nature, meaning its influence decreases with each time but is unbounded. $c$ represent the confidence level. We determine the action over,

$ \text{Action}(t)  = arg \max_{a} \left [ \text{Estimate}(t) + \text{upper bound} \right ]$

<img src="images\UCB-insight.png">

## Code and analysis